In [1]:
import os
import xarray as xr
import rasterio as rio

In [2]:
def get_imgs_and_profile(imgs_path):
    
    tci_images = []
    profile = None
    
    for root, dirs, files in os.walk(imgs_path, topdown=True):
        for fname in files:
            if fname.endswith('_TCI.jp2'):
                if profile is None:
                    temp_img = rio.open(os.path.join(root, fname))
                    profile = temp_img.profile
                tci_images.append(xr.open_rasterio(os.path.join(root, fname) , chunks={'x': 1000, 'y': 1000}))
    return (tci_images,profile)


##########################################
##########################################


def get_median_img(imgs, no_of_bands):
    
    bands_medians = []
    for b in range(no_of_bands):
        bands = [img.sel(band=b+1) for img in imgs]
        bands_comp = xr.concat(bands, dim='band')
        bands_medians.append(bands_comp.median(dim='band', skipna=True))
    
    return xr.concat(bands_medians,dim='band')

In [3]:
imgs, imgs_profile = get_imgs_and_profile('./sentinel2-imgs/')

imgs[0]

<xarray.DataArray (band: 3, y: 10980, x: 10980)>
dask.array<open_rasterio-ddb9f3f1454bc8a19040788eec213377<this-array>, shape=(3, 10980, 10980), dtype=uint8, chunksize=(3, 1000, 1000), chunktype=numpy.ndarray>
Coordinates:
  * band     (band) int64 1 2 3
  * y        (y) float64 3.4e+06 3.4e+06 3.4e+06 ... 3.29e+06 3.29e+06 3.29e+06
  * x        (x) float64 3e+05 3e+05 3e+05 ... 4.098e+05 4.098e+05 4.098e+05
Attributes:
    transform:   (10.0, 0.0, 300000.0, 0.0, -10.0, 3400020.0)
    crs:         +init=epsg:32636
    res:         (10.0, 10.0)
    is_tiled:    1
    nodatavals:  (nan, nan, nan)
    scales:      (1.0, 1.0, 1.0)
    offsets:     (0.0, 0.0, 0.0)

In [4]:
median_img = get_median_img(imgs, 3)

median_img

<xarray.DataArray (band: 3, y: 10980, x: 10980)>
dask.array<concatenate, shape=(3, 10980, 10980), dtype=float64, chunksize=(1, 5000, 5000), chunktype=numpy.ndarray>
Coordinates:
  * y        (y) float64 3.4e+06 3.4e+06 3.4e+06 ... 3.29e+06 3.29e+06 3.29e+06
  * x        (x) float64 3e+05 3e+05 3e+05 ... 4.098e+05 4.098e+05 4.098e+05
Dimensions without coordinates: band

In [5]:
with rio.Env():
    imgs_profile.update(driver='GTiff')
    with rio.open('./output/medians.tif', 'w', **imgs_profile) as dst:
        dst.write(median_img.astype(rio.uint8))